In [ ]:
import pandas as pd

## 오프라인 세일즈 데이터 -> 매장별 개폐업 일자 데이터로 변환 

In [18]:
import pandas as pd

# 데이터프레임 로드
offline_sales = pd.read_csv('../assets/data/offline_offline_wkly_Hazzys_sales.csv')

# 오프라인 스토어별 개장일(Open_Store_Week0)과 폐업일(Close_Store_Week0)을 추출하여 새로운 데이터프레임 생성
offline_store_info = offline_sales[['Off.Store.Code', 'Off.Store.Name2', 'Open_Store_Week0', 'Close_Store_Week0']]

# 중복 제거를 위해 고유한 매장 정보만 추출
offline_store_info = offline_store_info.drop_duplicates()

# 새로운 데이터프레임 확인
print(offline_store_info['Close_Store_Week0'].unique())

[202112.     nan 202233. 202252. 202208. 202122. 202234. 202108. 201942.
 202121. 202127. 202230. 202134. 202011. 202203. 202144. 202023. 202107.
 202102. 202113. 202035. 202032. 202309. 201909. 202008. 202013. 202018.
 201911. 202020. 201926. 202009. 201947. 202017. 202215. 202043. 201950.
 202031. 201908. 202326. 202010. 202211. 202220. 202037. 201917. 202130.
 202052. 202039. 201903. 202110. 202114. 201938. 202104. 201922. 201945.
 201912. 201905. 201939. 201935. 202352. 201934. 202024. 201921. 202319.
 202040. 202003. 201904. 202205. 202135. 202236. 202226. 201914. 202325.
 202350. 201931. 202132. 201932. 202321. 201916. 201915. 201930. 201902.
 202315. 202322. 202351. 202120. 201918. 202005. 202117. 202015. 202038.
 202046. 202348. 202111. 202235. 202029. 202124. 202335. 202125. 202331.
 202250. 202231. 202223. 202004. 202045. 202027. 202241. 202119. 202049.
 202139. 202340. 202006. 202304. 202224. 202225. 202228. 202305. 202116.
 202036. 201943. 202019. 202033. 202042. 202050. 20

In [5]:
online_Wkly_HZ = pd.read_csv('../assets/data/online_Wkly.HZ.csv')

# Wk_Year별로 Sales.Actual의 합산을 구하여 온라인 매출 시계열 데이터 생성
online_sales_by_year = online_Wkly_HZ.groupby('Wk_Year')['Sales.Actual'].sum().reset_index()

# 데이터 확인
print(online_sales_by_year.head())

   Wk_Year   Sales.Actual
0   201901  219852.685182
1   201902  107737.669109
2   201903   92954.728918
3   201904  110051.737891
4   201905   83874.081573


In [14]:
print(online_sales_by_year.tail())

     Wk_Year   Sales.Actual
256   202348  119510.410282
257   202349  106832.188627
258   202350  112875.840000
259   202351   98747.958227
260   202352   81682.452045


In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 개업일과 폐업일에 대한 이벤트 생성
events = []

# 개업 이벤트 추가
for idx, row in offline_store_info.iterrows():
    events.append({'date': row['Open_Store_Week0'], 'event': 'Open', 'store': row['Off.Store.Name2']})

# 폐업 이벤트 추가
for idx, row in offline_store_info.iterrows():
    if pd.notna(row['Close_Store_Week0']):  # 폐업일이 존재하는 경우에만 추가
        events.append({'date': row['Close_Store_Week0'], 'event': 'Close', 'store': row['Off.Store.Name2']})

# 이벤트 데이터프레임 생성
events_df = pd.DataFrame(events)

# 주 번호를 날짜로 변환하는 함수
def week_to_date(year_week):
    year = int(str(year_week)[:4])  # 연도 추출
    week = int(str(year_week)[4:])   # 주 번호 추출
    return pd.to_datetime(f'{year}-W{week}-1')  # 해당 연도의 첫 번째 날(월요일)

# 이벤트 날짜를 datetime 형식으로 변환
events_df['date'] = events_df['date'].apply(week_to_date)

# 온라인 매출 데이터의 Wk_Year를 날짜로 변환
online_sales_by_year['Wk_Year'] = online_sales_by_year['Wk_Year'].apply(week_to_date)
# 이벤트와 온라인 매출 데이터 병합
merged_df = pd.merge_asof(events_df.sort_values('date'), 
                           online_sales_by_year.sort_values('Wk_Year'), 
                           left_on='date', 
                           right_on='Wk_Year', 
                           direction='backward')

# 매출 변화량 계산
merged_df['Sales.Change'] = merged_df['Sales.Actual'].diff()

# 개업과 폐업 이벤트에 따른 매출 변화량 시각화
plt.figure(figsize=(14, 7))
plt.plot(merged_df['date'], merged_df['Sales.Actual'], marker='o', label='Online Sales')
plt.axvline(x=merged_df['date'][merged_df['event'] == 'Open'].min(), color='g', linestyle='--', label='Store Openings')
plt.axvline(x=merged_df['date'][merged_df['event'] == 'Close'].min(), color='r', linestyle='--', label='Store Closures')

# 이벤트 표시
for i, row in merged_df.iterrows():
    plt.text(row['date'], row['Sales.Actual'], row['event'], fontsize=9, ha='center')

plt.title('Effect of Store Openings and Closures on Online Sales')
plt.xlabel('Date')
plt.ylabel('Sales Actual')
plt.xticks(rotation=45)
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

에러발생 event_df['date']: 0      200701.0
1      200701.0
2      200701.0
3      200701.0
4      200701.0
         ...   
744    202351.0
745    202043.0
746    202013.0
747    202326.0
748    202315.0
Name: date, Length: 749, dtype: float64, online_sales_by_year['Wk_Year']: 0      201901
1      201902
2      201903
3      201904
4      201905
        ...  
256    202348
257    202349
258    202350
259    202351
260    202352
Name: Wk_Year, Length: 261, dtype: int64


MergeError: incompatible merge keys [0] dtype('float64') and dtype('int64'), must be the same type